In [2]:
# IMPORTS
from pathlib import Path
import json
import pandas as pd
import numpy as np
import sys
import csv
from collections import defaultdict, Counter
import json
import shutil
from PIL import Image

In [14]:
# CONSTANTS
BASE_DIR = Path.cwd().parent
STATS_DIR = BASE_DIR / 'stats'
KINETICS_DIR = BASE_DIR / 'work_dir' / 'kinetics_700'
KINETICS_DIR = Path('/home/gitumarkk/Desktop/GITHUB/dance-datasets-downloader/resources/700')
GIF_DIR = Path('/home/gitumarkk/Desktop/WORK_DIR/kinetics_gif/train_180_16')
VIDEO_PATH = Path('/media/gitumarkk/Seagate Backup Plus Drive/Dancelogue/DATASETS/Kinetics')

DATA_PATH = Path('/home/gitumarkk/Desktop/WORK_DIR/kinetics_gif/train_180_16_json')
DATA_PATH.mkdir(exist_ok=True, parents=True)


In [10]:
def read_results_file(source):
    with source.open(mode='r') as results_json:
        data = json.load(results_json)
    return data

def read_kinetics_data():
    all_data = {}
    for f1 in KINETICS_DIR.iterdir():
        if f1.stem in ['train', 'val', 'test']:
            for f2 in f1.iterdir():
                if f2.suffix == '.json':
                    data = read_results_file(f2)
                    all_data.update(data)
    return all_data

kinetics_data = read_kinetics_data() 

In [11]:
print(len(kinetics_data.keys()))
kinetics_keys_dict = {key: True for key in kinetics_data.keys()}

646984


In [6]:
def get_gif_data():
    gif_dict = {}
    for f1 in GIF_DIR.iterdir():
        if f1.is_dir():
            for f2 in f1.iterdir():
                gif_dict[f2.stem] = f2
    return gif_dict

gif_data = get_gif_data()
                

In [7]:
print(len(gif_data.keys()))

532185


In [8]:
def move_gif_not_in_700():
    GIF_DIR_MISSING = GIF_DIR.parent / 'train_not_in_700'
    GIF_DIR_MISSING.mkdir(exist_ok=True, parents=True)
    
    m = 0
    for key in gif_data.keys():
        if not kinetics_keys_dict.get(key):
            na = gif_data[key]
            (GIF_DIR_MISSING / na.parent.stem).mkdir(exist_ok=True, parents=True)
            
            shutil.move(str(na), str(GIF_DIR_MISSING / na.parent.stem / na.name))
            print(gif_data[key])
            m += 1
    print(m)
    
move_gif_not_in_700()

0


In [9]:
def create_train_json():
    output = {}
    for f1 in GIF_DIR.parent.iterdir():
        if f1.stem in ['train_180_16']:
            for f2 in f1.iterdir():
                output[f2.stem] = []

                for f3 in f2.iterdir():
                    try:
                        img = Image.open(str(f3))
                        output[f2.stem].append({
                          'g': f3.name,
                          'w': img.width,
                          'h': img.height
                        })
                    except Exception as e:
                        print(e)

                with (DATA_PATH / "{}.json".format(f2.stem)).open(mode='w') as f2_json:
                    json.dump(output[f2.stem], f2_json)


    return

# create_train_json()

In [37]:
def get_video_data():
    gif_dict = {}
    for f1 in VIDEO_PATH.iterdir():
        if f1.stem in ['train', 'val', 'test']:
            for f2 in f1.iterdir():
                if f1.stem == 'test':
                    gif_dict[f2.stem] = {
                        'path': f2,
                        'split': f1.stem
                    }
                else:
                    for f3 in f2.iterdir():
                        gif_dict[f3.stem] = {
                            'path': f3,
                            'split': f1.stem,
                            'label': f2.stem
                        }
    return gif_dict

video_data = get_video_data()

In [38]:
print(len(video_data.keys()))

631604


In [39]:
def move_video_not_in_700():
    VIDEO_DIR_MISSING = VIDEO_PATH / 'train_not_in_700'
    
    for s in ['train', 'test', 'val']:
       (VIDEO_PATH / "{}_na".format(s)).mkdir(exist_ok=True, parents=True)
    #     GIF_DIR_MISSING.mkdir(exist_ok=True, parents=True)
    
    m = 0
    for key in video_data.keys():
        if not kinetics_keys_dict.get(key):
            na = video_data[key]['path']
            split = video_data[key]['split']
            label = video_data[key]['label']
            
            (VIDEO_PATH / "{}_na".format(split) / label).mkdir(exist_ok=True, parents=True)
            shutil.move(str(na), str(VIDEO_PATH / "{}_na".format(split) / label / na.name))
            
            m += 1
    print(m)
    
move_video_not_in_700()

0
